In [3]:
import pandas as pd
import re

In [ ]:
input_file = "D:\\University-Master\\Term-3\\Web Mining\\PRO\\Semantic Search Engine\\Data Collecting\\ganj_results_real.csv"
output_file = "D:\\University-Master\\Term-3\\Web Mining\\PRO\\Semantic Search Engine\\Data Cleaning\\output_parsed.csv"

print("Reading input CSV file...")
df = pd.read_csv(input_file)
df.head()

Reading input CSV file...


,id,meta_text,abs_text,keyword_text
0,1,پیش بینی تقاضای قطعات یدکی با استفاده از هوش م...,هوش مصنوعی و یادگیری ماشینی روزبه‌روز در حال ت...,هوش مصنوعی / هوش مصنوعی\nتولید قطعات / هوش مصنوعی
1,2,تاثیر همکاری انسان و هوش مصنوعی بر رفاه زنجیره...,همکاری انسان و هوش مصنوعی فرصت‌های قابل توجهی ...,همکاری انسان و هوش مصنوعی / اعتماد به هوش مصنو...
2,3,چالش های کاربردی هوش مصنوعی در دادرسی کیفری؛ ب...,کاربرد فناوری های نوین ناشی از هوش مصنوعی در د...,هوش مصنوعی. آیین دادرسی کیفری. دادرسی منصفانه ...
3,4,ارزیابی عوامل موثر بر ایجاد اعتماد مشتری به هو...,یکی از چالش‌های مهم در استفاده از هوش مصنوعی پ...,هوش مصنوعی انسان‌نما / رفتار مشتری\nاعتماد مشت...
4,5,امکان سنجی تحقق نفس انسانی در هوش مصنوعی بر اس...,در عصر حاضر، هوش مصنوعی بسیاری از ساحت های حیا...,نفس انسانی هوش مصنوعی امکان سنجی فلسفه اسلامی ...


In [5]:
print("Initial number of records:", len(df))

Initial number of records: 300


In [6]:
# Drop rows where 'meta_text' is completely empty or null
df = df.dropna(subset=['meta_text'])                  # Drop NaN
df = df[df['meta_text'].str.strip() != '']           # Drop empty strings

print("After dropping empty rows:", len(df))

After dropping empty rows: 300


In [7]:
# Drop completely duplicate rows based on 'meta_text'
df = df.drop_duplicates(subset=['meta_text'], keep='first')

print("After dropping duplicate rows:", len(df))

After dropping duplicate rows: 300


In [8]:
# ---------------------------------
# Split names safely (always returns list)
# ---------------------------------
def split_names(text):
    if not text:
        return []

    parts = re.split(r'[،,;]| و ', text)
    return [p.strip() for p in parts if p.strip()]


In [9]:
import re

def extract_contributors(contributors_line):

    authors = []
    advisors = []
    co_advisors = []

    # نرمال سازی فاصله‌ها
    contributors_line = re.sub(r"\s+", " ", contributors_line).strip()

    pattern = r"(پدیدآور|استاد راهنما|استاد مشاور):\s*(.*?)(?=\s*(پدیدآور|استاد راهنما|استاد مشاور):|$)"

    matches = re.findall(pattern, contributors_line)

    for role, names, _ in matches:
        names_list = split_names(names)

        if role == "پدیدآور":
            authors.extend(names_list)

        elif role == "استاد راهنما":
            advisors.extend(names_list)

        elif role == "استاد مشاور":
            co_advisors.extend(names_list)

    return authors, advisors, co_advisors

In [11]:
# ---------------------------------
# Smart position-based parser
# ---------------------------------
def parse_meta_position_based(meta_text):

    data = {
        'title': None,
        'doc_type': None,
        'degree': None,
        'year': None,
        'subject': None,
        'university': None,
        'authors': [],
        'advisors': [],
        'co_advisors': []
    }

    if not isinstance(meta_text, str):
        return data

    lines = [line.strip() for line in meta_text.split('\n') if line.strip()]

    # Basic fixed positions
    if len(lines) >= 4:
        data['title'] = lines[0]
        data['doc_type'] = lines[1]
        data['degree'] = lines[2]
        data['year'] = lines[3]

    # Scan remaining lines dynamically
    for line in lines[4:]:

        # Subject (optional)
        if line.startswith("موضوع:"):
            data['subject'] = line.replace("موضوع:", "").strip()

        # Contributors
        elif line.startswith("پدیدآور:"):
            authors, advisors, co_advisors = extract_contributors(line)
            data['authors'] = authors
            data['advisors'] = advisors
            data['co_advisors'] = co_advisors

        # University
        elif any(word in line for word in ["دانشگاه", "موسسه", "مؤسسه"]):
            data['university'] = line

    return data


In [12]:
print("Parsing meta_text column...")
parsed_df = df['meta_text'].apply(parse_meta_position_based).apply(pd.Series)

Parsing meta_text column...


In [13]:
# Convert list columns to string for CSV storage
for col in ['authors', 'advisors', 'co_advisors']:
    parsed_df[col] = parsed_df[col].apply(lambda x: ' | '.join(x) if x else None)

# Merge with original dataframe
df_final = pd.concat([df, parsed_df], axis=1)

In [14]:
# Drop the meta_text column if it exists
df_final = df_final.drop(columns=['meta_text'], errors='ignore')

# Columns to move to the end
last_cols = [col for col in ['abs_text','keyword_text','subject'] if col in df_final.columns]

# Reorder columns: all other columns first, then abs_text, keyword_text, subject
df_final = df_final[[c for c in df_final.columns if c not in last_cols] + last_cols]

df_final.head(10)

,id,title,doc_type,degree,year,university,authors,advisors,co_advisors,abs_text,keyword_text,subject
0,1,پیش بینی تقاضای قطعات یدکی با استفاده از هوش م...,پیشنهاده,کارشناسی ارشد,1402,دانشگاه کردستان، دانشکده فنی و مهندسی,رویا میرزائی مهر,محمود شهرخی,NaN,هوش مصنوعی و یادگیری ماشینی روزبه‌روز در حال ت...,هوش مصنوعی / هوش مصنوعی\nتولید قطعات / هوش مصنوعی,NaN
1,2,تاثیر همکاری انسان و هوش مصنوعی بر رفاه زنجیره...,پیشنهاده,کارشناسی ارشد,1404,دانشگاه شهید اشرفی اصفهانی، دانشکده مدیریت,رویا شهابی,حمیدرضا طلایی,NaN,همکاری انسان و هوش مصنوعی فرصت‌های قابل توجهی ...,همکاری انسان و هوش مصنوعی / اعتماد به هوش مصنو...,NaN
2,3,چالش های کاربردی هوش مصنوعی در دادرسی کیفری؛ ب...,پیشنهاده,کارشناسی ارشد,1401,دانشگاه تربیت مدرس، دانشکده حقوق,ملیحه ابوالقاسمی,روح الدین کردعلیوند,NaN,کاربرد فناوری های نوین ناشی از هوش مصنوعی در د...,هوش مصنوعی. آیین دادرسی کیفری. دادرسی منصفانه ...,NaN
3,4,ارزیابی عوامل موثر بر ایجاد اعتماد مشتری به هو...,پیشنهاده,کارشناسی ارشد,1402,دانشگاه علم و صنعت ایران، دانشکده مدیریت، اقتص...,بهراد سلطانی,یاسر سبحانی فرد,NaN,یکی از چالش‌های مهم در استفاده از هوش مصنوعی پ...,هوش مصنوعی انسان‌نما / رفتار مشتری\nاعتماد مشت...,NaN
4,5,امکان سنجی تحقق نفس انسانی در هوش مصنوعی بر اس...,پیشنهاده,دکتری تخصصی (PhD),1403,دانشگاه ادیان و مذاهب، دانشکده فلسفه,امیرحسین ایمانی مقدم,حسن معلمی | احمد لهراسبی,بهروز مینایی,در عصر حاضر، هوش مصنوعی بسیاری از ساحت های حیا...,نفس انسانی هوش مصنوعی امکان سنجی فلسفه اسلامی ...,NaN
5,6,بررسی تاثیر شایستگی های هوش مصنوعی(AI) بر عملک...,پیشنهاده,کارشناسی ارشد,1402,موسسه آموزش عالی گلستان، دانشکده علوم انسانی,عباس زرگری,سمانه میقانی,سمانه پهلوان,ای تحقیق به بررسی تاثیر شایستگی های هوش مصنوعی...,هوش مصنوعی / هوش مصنوعی\nقابلیت های بازاریابیB...,NaN
6,7,مطالعه تطبیقی مسئولیت مدنی ناشی از استفاده از ...,پیشنهاده,کارشناسی ارشد,1402,دانشگاه خاتم، دانشکده علوم انسانی,مهدیه فولادی,مصطفی نورالهی,NaN,هدف اصلی این تحقیق تحلیل چالش ها و مسائل حقوقی...,مطالعه تطبیقی ـ هوش مصنوعی ـ مسئولیت مدنی ـ حق...,NaN
7,8,فراتحلیل تاثیر هوش مصنوعی بر بهره وری منابع ان...,پیشنهاده,کارشناسی ارشد,1403,دانشگاه پیام نور استان خراسان جنوبی، مرکز پیام...,رضا برافشان,زهره شرعی,NaN,چکیده : در عصر فناوری دیجیتال، اتخاذ هوش مصنوع...,هوش مصنوعی، بهره وری منابع انسانی، فراتحلیل / ...,NaN
8,9,تأثیر یک ابزار مبتنی بر هوش مصنوعی بر گرامر و ...,پیشنهاده,کارشناسی ارشد,1404,دانشگاه پیام نور استان آذربایجان شرقی، مرکز پی...,مهسا تابستانی,مینا عباسی بنابی,مرتضی جعفری,اعتقاد بر این است که دستور زبان نقش اساسی در پ...,هوش مصنوعی_ یادگیری گرامر_برقراری ارتباط / AI-...,NaN
9,10,بررسی عدم قطعیّت عملکرد حوضچه های رسوبی با است...,پیشنهاده,کارشناسی ارشد,1401,دانشگاه تحصیلات تکمیلی صنعتی و فناوری پیشرفته ...,محمد جعفر جهرومی شیرازی,محمد نجف زاده,NaN,حوضچه های ترسیب نقش جداکننده ذرّات معلق قابل ت...,3-مدل های هوش مصنوعی / 3-مدل های هوش مصنوعی\nذ...,NaN


In [15]:
print("Saving output file...")
df_final.to_csv(output_file, index=False, encoding='utf-8-sig')

print("Processing completed successfully.")
print(f"Output file saved as: {output_file}")

Saving output file...
Processing completed successfully.
Output file saved as: D:\University-Master\Term-3\Web Mining\PRO\Test\Data Cleaning\output_parsed.csv
